In [1]:
import numpy as np
import tensorflow as tf
import random
import sys
import eli5

import d2d
import importlib
importlib.reload(d2d)
import pandas as pd
import os

2022-11-15 23:06:44.841644: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 23:06:44.988200: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-15 23:06:45.021542: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 23:06:45.668185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:
history = {}
val_performance = {}
performance = {}

seeds = list(np.arange(1,26,1))
input_columns = [list(np.arange(0, 2308, 1)), list(np.arange(0, 1600, 1)), list(np.arange(1600, 2308, 1))]
location = ['Whole Cable', 'Ablation Zone', 'Accumulation Zone']

for SEED in seeds:
    
    for ic, loc in zip(input_columns, location):
        

        file = "/data/fast0/datasets/Rhone_data_continuous_highpass.h5"

        filt = 'Highpass'

        ww = 200
        bs = 32

        random.seed(SEED)
        np.random.seed(SEED)
        tf.random.set_seed(SEED)


        linear_model, lstm_model, dnn_model, df_all_chan, das_data_all, f  = d2d.import_data(filename = file, input_columns = ic)

        da = df_all_chan.to_numpy()

        multi_step_window_shuffled = d2d.WindowGenerator(df_all_chan,
                                                        input_width=ww,
                                                        label_width=1, 
                                                        shift=0,
                                                        label_columns=['Discharge'],
                                                        input_columns=ic,
                                                        shuffle=True,
                                                        batch_size=bs)

        history['lstm '+loc+' '+str(SEED)] = d2d.compile_and_fit(lstm_model, multi_step_window_shuffled, learning_rate = 0.001)

        val_performance['lstm '+loc+' '+str(SEED)] = lstm_model.evaluate(multi_step_window_shuffled.val)
        performance['lstm '+loc+' '+str(SEED)] = lstm_model.evaluate(multi_step_window_shuffled.test, verbose=0)


In [ ]:
mae_all = []
for i in performance.values():
    mae_all.append(i[-1])

In [ ]:
whole_cable = mae_all[0::3]
ablation_zone = mae_all[1::3]
accumulation_zone = mae_all[2::3]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(7,7))
fig.patch.set_facecolor('white')

ax.hist(whole_cable, label='Whole Cable Length', lw=2, alpha = 0.7)
ax.hist(ablation_zone, label='Ablation Zone', lw=2, alpha = 0.7)
ax.hist(accumulation_zone, label='Accumulation Zone', lw=2, alpha = 0.7)
ax.set_ylabel('Frequency')
ax.set_xlabel('Mean Absolute Error')
ax.legend()